In [12]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import nltk
import mysql.connector
import ssl

# konksi nanti coba ditaruh diluar
# koneksi
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='UnNgGrape'
    )
cursor = db.cursor()

class Scrape:

    def scrapeMain(keyword):
        # inisialisasi array untuk menampung hasil data
        asal_situs = []
        title_lowongan = []
        nama_perusahaan = []
        lokasi_perusahaan = []
        keterangan_lowongan = []
        skill_lowongan = []
        benefit_lowongan = []
        deskripsi_lowongan = []
        stem_detail = []
        link_lowongan = []

        # pake try except (continue) kalau udah looping keyword
        # loop scrape
        cursor.execute("SELECT * FROM `scrape` WHERE `asal_situs` LIKE 'karir'")
        raw_scrape = cursor.fetchall()
        for rowS in raw_scrape:
            asal = rowS[0]
            # scrape data
            main_link1 = rowS[1]
            main_link2 = rowS[2]
            main_link = main_link1 + keyword + main_link2
            tag_main = rowS[3]
            tag_lowongan = rowS[4]
            tag_lowongan_part = rowS[5]
            tag_perusahaan = rowS[6]
            tag_perusahaan_part = rowS[7]
            tag_lokasi = rowS[8]
            tag_lokasi_part = rowS[9]
            tag_mainDetail = rowS[10]
            tag_keterangan = rowS[11]
            tag_skill = rowS[12]
            tag_benefit = rowS[13]
            tag_deskripsi = rowS[14]
            breakDeskripsi = rowS[15]
            raw_link = rowS[16]

            # tab sekali jika sudah masuk looping scrape
            r = Request(main_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
            response = urlopen(r).read()
            soup = BeautifulSoup(response, "lxml")
            print(type(soup))

            jobList = soup.find_all("div", tag_main)
            for p in jobList:
                # link scrape dulu untuk cek data sudah pernah di scrape atau belum
                link = raw_link+p.find('a').get('href')
                lowongan = p.find(tag_lowongan_part, tag_lowongan).get_text()
                # menggunakan try except karena ada beberapa perusahaan yang dirahasiakan
                try:
                    perusahaan = p.find(tag_perusahaan_part, tag_perusahaan).get_text()
                except:
                    perusahaan = "Perusahaan Dirahasiakan"

                try:
                    lokasi = p.find(tag_lokasi_part, tag_lokasi).get_text().replace("head office - ", "")
                except Exception as e:
                    lokasi = str(e)

                print(link)

                # try print return scrapeDetail
                keterangan, skill, benefit, deskripsi = Scrape.scrapeDetail(link, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi)

                # stemming for detail
                raw_stem = lowongan + " " + perusahaan + " " + lokasi + " " + str(keterangan) + " " + str(skill) + " " + str(benefit) + " " + str(deskripsi)
                stem = Scrape.stemming(raw_stem)

                # push to array penampung hasil data
                asal_situs.append(asal)
                title_lowongan.append(lowongan)
                nama_perusahaan.append(perusahaan)
                lokasi_perusahaan.append(lokasi)
                keterangan_lowongan.append(keterangan)
                skill_lowongan.append(skill)
                benefit_lowongan.append(benefit)
                deskripsi_lowongan.append(deskripsi)
                stem_detail.append(stem)
                link_lowongan.append(link)

        # tampil pakai panda
        jobList_dict ={'asal':asal_situs, 'lowongan':title_lowongan, 'perusahaan':nama_perusahaan, 'lokasi':lokasi_perusahaan, 'keterangan':keterangan_lowongan, 'skill':skill_lowongan, 'benefit':benefit_lowongan, 'deskripsi':deskripsi_lowongan, 'stem':stem_detail, 'link':link_lowongan}
        df = pd.DataFrame(jobList_dict,columns = ['asal', 'lowongan', 'perusahaan', 'lokasi', 'keterangan', 'skill', 'benefit', 'deskripsi', 'stem', 'link'])
        return df.sort_values('asal',ascending=True)

    def scrapeDetail(linkDetail, tag_mainDetail, tag_keterangan, tag_skill, tag_benefit, tag_deskripsi, breakDeskripsi):
        r = Request(linkDetail, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'})
        response = urlopen(r).read()
        soup = BeautifulSoup(response, "lxml")

        raw_detail = soup.find_all("div", tag_mainDetail)
        for p in raw_detail:
            # menggunakan try except karena ada beberapa estimasi keterangan yang tidak terlampir
            try:
                raw_keterangan = p.find("div", tag_keterangan ).get_text(separator=". ").replace("Fungsi Kerja. ", "")
                nltk_tokens = nltk.sent_tokenize(raw_keterangan)
                keterangan = ""

                for x in nltk_tokens:
                    if x == "Lamar.":
                        break
                    keterangan = keterangan + " " + x
            except:
                keterangan = "-"
            
            try:
                skill = p.find("div", tag_skill).get_text(separator=" ")
            except:
                skill = "-"
            try:
                benefit = p.find("div", tag_benefit).get_text(separator=" ").replace("Tunjangan dan keuntungan", "keuntungan:")
            except:
                benefit = "-"

            # menggunakan try except karena ada beberapa deskripsi yang NoneType
            try:
                # replace untuk glints
                raw_deskripsi = p.find("div", tag_deskripsi ).get_text(separator=". ").replace("Informasi Penting. Pastikan perusahaan yang kamu lamar resmi dengan memeriksa website dan lowongan kerja mereka.. Read Less.", "")
                nltk_tokens = nltk.sent_tokenize(raw_deskripsi)
                deskripsi = ""

                for x in nltk_tokens:
                    if x == breakDeskripsi:
                        break
                    deskripsi = deskripsi + " " + x
            except:
                deskripsi = "-"
            
            # give to scrapeMain
            return keterangan, skill, benefit, deskripsi

    def stemming(raw_stem):
        # define punctuation
        punctuations = '''!()-[]{};:=+`'",<>./|\?@#$%^&*_~'''

        # remove punctuation from the string
        no_punct = ""
        for char in raw_stem:
            if char not in punctuations:
                no_punct = no_punct + char
            else:
                no_punct = no_punct + " "

        # display the unpunctuated string
        raw_stem = no_punct.lower()

        # create stemmer
        # nltk
        ps = PorterStemmer()
        # sastrawi
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()

        # stemming process
        # nltk
        words = word_tokenize(raw_stem)
        nltk_stemmer = ""
        for w in words:
            nltk_stemmer = nltk_stemmer + " " + ps.stem(w)
        # sastrawi sekaligus return
        return stemmer.stem(nltk_stemmer)

# print(Scrape.getKeyword())
# print("\n")
# Scrape.scrapeMain() 
Scrape.scrapeMain("palembang")

<class 'bs4.BeautifulSoup'>
https://karir.com/opportunities/1323308
https://karir.com/opportunities/1323795
https://karir.com/opportunities/1323419
https://karir.com/opportunities/1321525
https://karir.com/opportunities/1317769
https://karir.com/opportunities/1322924
https://karir.com/opportunities/1317672
https://karir.com/opportunities/1322914
https://karir.com/opportunities/1317767
https://karir.com/opportunities/1220357
https://karir.com/opportunities/1265976


,asal,lowongan,perusahaan,lokasi,keterangan,skill,benefit,deskripsi,stem,link
0,karir,SALES - Palembang,PT Ekadharma International,Jakarta Utara,"Penjualan (Teknis), Penjualan (Non-Teknis), P...",-,-,Persyaratan. Usia Max. 35 tahun \r\n. Pendidi...,sale palembang pt ekadharma intern jakarta uta...,https://karir.com/opportunities/1323308
1,karir,"Field Collection Area Palembang, Baturaja & Pr...",PT Home Credit Indonesia,Palembang - Palembang,Kolektor. Jenjang Karir. Pemula / Staf. Penga...,-,-,Persyaratan. Fresh Graduate\r\n. Usia 21- 45 ...,field collect area palembang baturaja prabumul...,https://karir.com/opportunities/1323795
2,karir,SALES EXECUTIVE BAKERY & FSR PALEMBANG,PT Nusa Talenta Indonesia,DELISARI PALEMBANG - Palembang,"Pemasaran (Non-Teknis), Penjualan (Non-Teknis...",-,-,Persyaratan. KUALIFIKASI :. 1. PRIA / WANITA ...,sale execut bakeri fsr palembang pt nusa talen...,https://karir.com/opportunities/1323419
3,karir,FIELD AGENT PALEMBANG,Confidential Company,Kota Palembang - Palembang,Kolektor. Jenjang Karir. Pemula / Staf. Penga...,-,-,Persyaratan. Syarat pelamar:\r\n. . Usia maks...,field agent palembang confidenti compani kota ...,https://karir.com/opportunities/1321525
4,karir,Teknisi Elektronik area Palembang,PT Yongwang Electronics Indonesia,Palembang - Palembang,"Teknik, Perangkat Keras. Jenjang Karir. Pemul...",-,-,Persyaratan. Kualifikasi\r\n. . Memiliki SIM ...,teknisi elektronik area palembang pt yongwang ...,https://karir.com/opportunities/1317769
5,karir,Trainee Manager - Based Palembang,PT Rekso Nasional Food (McDonald's Indonesia),Palembang - Palembang,Operasi Restoran. Jenjang Karir. Pemula / Sta...,-,-,Persyaratan. Cara Melamar:\r\n. . Ikuti pandu...,traine manag base palembang pt rekso nasion fo...,https://karir.com/opportunities/1322924
6,karir,Salesman Palembang,PT Yongwang Electronics Indonesia,Palembang - Palembang,Penjualan dan Pemasaran. Jenjang Karir. Pemul...,-,-,Persyaratan. Kandidat wajib memiliki pengalam...,salesman palembang pt yongwang electron indone...,https://karir.com/opportunities/1317672
7,karir,Crew Restoran - Palembang,PT Rekso Nasional Food (McDonald's Indonesia),Palembang - Palembang,Operasi Restoran. Jenjang Karir. Pemula / Sta...,-,-,Persyaratan. Cara Melamar:\r\n. . Ikuti pandu...,crew restoran palembang pt rekso nasion food m...,https://karir.com/opportunities/1322914
8,karir,Merchandiser Elektronik Palembang,PT Yongwang Electronics Indonesia,Palembang - Palembang,Penjualan dan Pemasaran. Jenjang Karir. Pemul...,-,-,Persyaratan. Kualifikasi:\r\n. . Diutamakan y...,merchandis elektronik palembang pt yongwang el...,https://karir.com/opportunities/1317767
9,karir,Crew Store Palembang,PT. Trimega Citra Nusantara,Palembang - Palembang,Penjualan dan Pemasaran. Jenjang Karir. Pemul...,-,-,Persyaratan. 1. Pria / Wanita single. . 2. Us...,crew store palembang pt trimega citra nusantar...,https://karir.com/opportunities/1220357
